# Matching Socks in Dark Room

### From a @CutTheKnotMath (Alexander Bogololny) post

The colors of ten pairs of socks range through different shades of gray from medium gray through black. Washed and dried in a dark room they have to be matched. Two socks form an unacceptable pair if their colors differ by more than one shade.

What is the probability of having 10 acceptable pairs when they are being matched - due to the light conditions -at random?

In [1]:
import math as math
import numpy as np
import pandas as pd

## Quantifying the problem

In [2]:
n0=10

In [3]:
socks0=list(range(1,n0+1))+list(range(1,n0+1))

Numbered socks, 1 to n, two of each; acceptable: numbers are within a distance of 1. Ideal match: k and k; k and k-1 or k and k+1 are acceptable.

In [4]:
socks0

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

## Generating permutations

### Non recursive version of Heap's algorithm

In [5]:
def permut(n,a):
    perms=[a[:]]
    c=[0 for i in range(n)]
    if n==1:
        return perms
    else:
        i=0
        while (i<n):
            if c[i]<i:
                if i%2==0:
                    tmp=a[0]
                    a[0]=a[i]
                    a[i]=tmp
                else:
                    tmp=a[c[i]]
                    a[c[i]]=a[i]
                    a[i]=tmp
                perms.append(a[:])
                c[i]=c[i]+1
                i=0
            else:
                c[i]=0
                i=i+1
    return perms

In [6]:
def partit(a):
    return [a[j:j+2] for j in range(0,len(a),2)]

In [7]:
def distp(p):
    return abs(p[1]-p[0])

In [8]:
def dista(a):
    return [distp(p) for p in a]

In [9]:
def accept(d):
    if max(d)<=1:
        return 1
    else:
        return 0

In [10]:
def multin(n):
    return math.factorial(2*n)//(math.factorial(2)**n)

We will find that we can divide our results by $(2^n)(n!)$, where the $n!$ stands for the permutations of the pairs (it doesn't matter the order of the pairs) and the $2^n$ means that that it doens't matter the order of the socks within each pair.

In [11]:
def naccept(n):
    fn=math.factorial(n)*(2**n)
    mn=multin(n)
    socks=list(range(1,n+1))+list(range(1,n+1))
    permsocks=permut(2*n,socks)
    permpairs=[partit(x) for x in permsocks]
    distsperms=[dista(x) for x in permpairs]
    acceptperms=[accept(x) for x in distsperms]
    sacc=sum(acceptperms)
    lacc=len(acceptperms)
    return [n,mn,sacc,lacc,sacc//fn,lacc//fn]

In [12]:
def acceptperm(a):
    return accept(dista(partit(a)))

In [13]:
tbl=[naccept(n) for n in range(2,6)]

Columns below:

n <br>
Multinomial(2,2,...,2): n times 2 <br>
Acceptable permutations <br>
Total permutations <br>
Acceptable reduced permutations <br>
Total reduced permutations

In [14]:
tbl

[[2, 6, 24, 24, 3, 3],
 [3, 90, 240, 720, 5, 15],
 [4, 2520, 4224, 40320, 11, 105],
 [5, 113400, 80640, 3628800, 21, 945]]

Stop now - too many permutations to list.

Should we rewrite our function to stop storing the permutations?
Or try to rewrite the permutations algorithm to account for the $(2^n)(n!)$ redundancies?

## Tangent: Number of ways of sitting m people in a row with n chairs wthout any neighbours

There are n ways with m=1 (including n=1) <br>
For $n>2$ and $m>1$, we can use the recurrence: <br>
$f(n,m)=\sum\limits_{k=2}^{n-1} f(n-k,m-1)$ 

Why? Choose a chair for the first person, take away everything to the left, the choice and the chair to the right; then use the previous results for m-1.

In [35]:
def f(n,m):
    if ((n==m) and (n>1)) or (n<m):
        return 0
    else:
        if m==1:
            return n
        else:
            sumf=0
            for k in range(2,n):
                sumf=sumf+f(n-k,m-1)
    return sumf

In [36]:
chairs=[[f(n,m) for n in range(1,11)] for m in range(1,11)]

In [31]:
dfchairs=pd.DataFrame(np.array(chairs),columns=range(1,11),index=range(1,11))

In [38]:
dfchairs.replace(0,'')

,1,2,3,4,5,6,7,8,9,10
1,1,2,3,4,5,6,7,8,9,10
2,,,1,3,6,10,15,21,28,36
3,,,,,1,4,10,20,35,56
4,,,,,,,1,5,15,35
5,,,,,,,,,1,6
6,,,,,,,,,,
7,,,,,,,,,,
8,,,,,,,,,,
9,,,,,,,,,,
10,,,,,,,,,,


## Back to our original program

Either all socks are perfectly matched, or socks are matched with neighbours. If a pair is matched, it cannot be used with the neighbours.

So you could have no pairs (exact matching), one pair, two pairs, ..., how many pairs?

For odd n the maximum number of pairs is (n-1)/2 <br>
For even n the maximum number of pairs is n/2

And the answers can be found on our chairs table, looking at n-1 and m.

For each desired number of pairs, look at $f(n-1,m)$, and consider $2^m$ ways of matching these pairs (for each m).

In [50]:
def g(n):
    sumg=1
    for m in range(1,n):
        sumg=sumg+f(n-1,m)*(2**m)
    return sumg

In [52]:
[g(n) for n in range(2,11)]

[3, 5, 11, 21, 43, 85, 171, 341, 683]

The total number of possibilities is:

In [55]:
def u(n):
    return math.factorial(2*n)//(math.factorial(n)*(2**n))

In [56]:
[u(n) for n in range(2,11)]

[3, 15, 105, 945, 10395, 135135, 2027025, 34459425, 654729075]

## And the answer is:

$$\frac{683}{654729075}$$